In [3]:
import torch
import torch.nn as nn
import torch.optim as opt
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Subset
import numpy as np
import onnx
import torch.nn.functional as F
import csv

# Constants
INPUT_DIM = 784  # after flattening
OUTPUT_DIM = 10
DATASET_DIR = '../dataset'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data loading and transformations
transform = tr.Compose([
    tr.ToTensor(),
    tr.Lambda(lambda x: torch.flatten(x))  # Flatten the image
])

train_dataset = torchvision.datasets.MNIST(DATASET_DIR, train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(DATASET_DIR, train=False, download=True, transform=transform)

# Creating subsets and data loaders
train_subset = Subset(train_dataset, indices=np.arange(4000))
train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


class SmallNetwork(nn.Module):
    def __init__(self, architecture):
        super(SmallNetwork, self).__init__()
        layers = []
        input_size = INPUT_DIM

        for i, layer_size in enumerate(architecture):
            layers.append(nn.Linear(input_size, layer_size))
            layers.append(nn.ReLU())
            input_size = layer_size

        layers.append(nn.Linear(input_size, OUTPUT_DIM))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


def train_mnist_network(architecture, folder_path):
    print(f"Architecture: {architecture}")
    print(f"Device: {device}")

    # Building the network
    model = SmallNetwork(architecture).to(device)

    # Initialization of the training parameters
    learning_rate = 0.001

    epochs = 1

    optimizer = opt.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    # Training loop
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        train_loss = running_loss / len(train_loader)
        train_accuracies.append(train_accuracy)
        train_losses.append(train_loss)

        # Evaluate on test set
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = 100 * correct / total
        test_loss = test_loss / len(test_loader)
        test_accuracies.append(test_accuracy)
        test_losses.append(test_loss)

        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

    # Save the model in ONNX format
    dummy_input = torch.randn(1, INPUT_DIM, device=device)
    onnx_path = f"{folder_path}/baseline_{architecture[0]}.onnx"
    torch.onnx.export(model, dummy_input, onnx_path)
    print(f"Model saved to {onnx_path}")

    return {
        "architecture": architecture,
        "train_accuracy": train_accuracies[-1],
        "test_accuracy": test_accuracies[-1],
        "train_loss": train_losses[-1],
        "test_loss": test_losses[-1],
    }


# Writing results to CSV file
csv_file = "accuracies_losses.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Architecture", "Train Accuracy", "Test Accuracy", "Train Loss", "Test Loss"])

    architectures = [ [200]]

    for architecture in architectures:
        folder_path = "output_folder"
        results = train_mnist_network(architecture, folder_path)
        print(results)

        # Write the architecture, accuracies, and losses to the CSV file
        writer.writerow([architecture, results['train_accuracy'], results['test_accuracy'],
                         results['train_loss'], results['test_loss']])

print(f"Results saved to {csv_file}")

Architecture: [7]
Device: cuda
Epoch [1/1], Train Loss: 2.1641, Test Loss: 2.0082, Train Accuracy: 24.18%, Test Accuracy: 35.75%
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



FileNotFoundError: [Errno 2] No such file or directory: 'output_folder/baseline_7.onnx'